In [ ]:
'''
first Set Up MySQL Database
-create a MySQL database using MySQL Workbench and set up the necessary tables.

Create Database:
.Open MySQL Workbench.
.Connect to your MySQL server.
.Execute the following SQL command to create a new database
'''

In [ ]:
#sql commands:

CREATE DATABASE restaurant_db;
USE restaurant_db;

#Create Tables:
CREATE TABLE orders (
    order_id INT PRIMARY KEY AUTO_INCREMENT,
    total DECIMAL(10, 2),
    payment_method VARCHAR(50),
    status VARCHAR(50)
);

CREATE TABLE order_items (
    item_id INT PRIMARY KEY AUTO_INCREMENT,
    order_id INT,
    item_name VARCHAR(255),
    item_price DECIMAL(10, 2),
    category VARCHAR(50),
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);
ALTER TABLE orders ADD COLUMN table_number INT;
ALTER TABLE order_items ADD COLUMN table_number INT;



In [ ]:
'''
Install MySQL Connector for Python
You'll need the mysql-connector-python library to interact with the MySQL database from Python.
'''
pip install mysql-connector-python

In [ ]:
import mysql.connector
#MenuItem Class: Represents an item on the menu with a name, price, and category.
class MenuItem:
    def __init__(self, name, price, category):
        self.name = name
        self.price = price
        self.category = category

    def __str__(self):
        return f"{self.name} - {self.category}: ${self.price:.2f}"

In [ ]:
class Order:
    _instance = None
    order_counter = 1

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(Order, cls).__new__(cls)
        return cls._instance

    def __init__(self):
        self.order_id = Order.order_counter
        self.items = []
        self.status = "Pending"
        self.payment = None
        self.table_number = None  
        Order.order_counter += 1

    def add_item(self, item):
        self.items.append(item)

    def remove_item(self, item):
        if item in self.items:
            self.items.remove(item)

    def calculate_total(self):
        return sum(item.price for item in self.items)

    def save_to_db(self):
        # Connect to MySQL
        conn = mysql.connector.connect(
            host='localhost',
            user='your_actual_username',  # Replace with your actual MySQL username
            password='',  # Empty string if no password
            database='restaurant_db'
        )
        cursor = conn.cursor()
        
        cursor.execute('''INSERT INTO orders (total, payment_method, status, table_number)
                          VALUES (%s, %s, %s, %s)''',
                       (self.calculate_total(), self.payment.payment_method, self.status, self.table_number))

        # Get the last inserted order_id
        self.order_id = cursor.lastrowid

        for item in self.items:
            cursor.execute('''INSERT INTO order_items (order_id, item_name, item_price, category,table_number)
                              VALUES (%s, %s, %s, %s,%s)''',
                           (self.order_id, item.name, item.price, item.category,self.table_number))
        
        conn.commit()
        conn.close()

    def make_payment(self, payment_method):
        self.payment = Payment(self, payment_method)
        payment_result = self.payment.process_payment()
        if self.payment.status == "Completed":
            self.status = "Paid"
            self.save_to_db()
        return payment_result

    def generate_receipt(self):
        if self.payment and self.payment.status == "Completed":
            item_list = "\n".join([str(item) for item in self.items])
            return (f"Receipt - Order ID: {self.order_id}\n"
                    f"Items:\n{item_list}\n"
                    f"Total: ${self.calculate_total():.2f}\n"
                    f"Payment Method: {self.payment.payment_method}\n"
                    f"Payment Status: {self.payment.status}\n"
                    f"Table Number: {self.table_number}\n")  # Add table number to the receipt
                    
        else:
            return "Payment not completed. Cannot generate receipt."

    def __str__(self):
        item_list = "\n".join([str(item) for item in self.items])
        return f"Order ID: {self.order_id}\nTable Number: {self.table_number}\nItems:\n{item_list}\nTotal: ${self.calculate_total():.2f}\nStatus: {self.status}"


In [ ]:
class Table:
    def __init__(self, table_number):
        self.table_number = table_number
        self.current_order = None
        self.is_occupied = False

    def assign_order(self, order):
        self.current_order = order
        self.is_occupied = True

    def clear_table(self):
        self.current_order = None
        self.is_occupied = False

    def __str__(self):
        status = "Occupied" if self.is_occupied else "Available"
        return f"Table {self.table_number} - {status}"

In [ ]:
#Once the order is ready, the staff member serves the order, marking it as "Completed" and freeing up the table.
class Staff:
    def __init__(self, name, role):
        self.name = name
        self.role = role

    def take_order(self, table, order):
        table.assign_order(order)

    def serve_order(self, table):
        if table.current_order:
            table.current_order.status = "Completed"
            print(f"Order {table.current_order.order_id} has been served.")
            table.clear_table()

    def __str__(self):
        return f"{self.name} - {self.role}"

In [ ]:
class Restaurant:
    def __init__(self, name):
        self.name = name
        self.menu = []
        self.tables = []
        self.staff = []

    def add_menu_item(self, item):
        self.menu.append(item)

    def remove_menu_item(self, item):
        if item in self.menu:
            self.menu.remove(item)

    def add_table(self, table):
        self.tables.append(table)

    def add_staff(self, staff_member):
        self.staff.append(staff_member)

    def assign_table(self, table_number, order):
        for table in self.tables:
            if table.table_number == table_number and not table.is_occupied:
                table.assign_order(order)
                return f"Order assigned to table {table_number}."
        return f"Table {table_number} is not available."

    def get_table_status(self, table_number):
        for table in self.tables:
            if table.table_number == table_number:
                return str(table)
        return "Table not found."

    def find_menu_item_by_name(self, item_name, order):
        for item in self.menu:
            if item.name.lower() == item_name.lower():  
                order.add_item(item)  
                print(f"Added {item_name} to the order.")
                return item
        print(f"Item '{item_name}' not found in the menu.")
        return None
    
    
    def process_payment(self, table_number, payment_method):
        table = next((t for t in self.tables if t.table_number == table_number), None)
        if table and table.current_order:
            return table.current_order.make_payment(payment_method)
        return "Table or order not found."

    def print_receipt(self, table_number):
        table = next((t for t in self.tables if t.table_number == table_number), None)
        if table and table.current_order:
            return table.current_order.generate_receipt()
        return "Table or order not found."

    
    
    def __str__(self):
        tables_status = "\n".join([str(table) for table in self.tables])
        return f"{self.name}\nTables:\n{tables_status}"

In [ ]:
class Payment:
    def __init__(self, order, payment_method):
        self.order = order
        self.payment_method = payment_method
        self.amount = order.calculate_total()
        self.status = "Pending"

    def process_payment(self):
        if self.payment_method in ["Cash", "Credit Card", "Digital Payment"]:
            self.status = "Completed"
            return f"Payment of ${self.amount:.2f} processed via {self.payment_method}."
        else:
            self.status = "Failed"
            return "Payment method not recognized. Payment failed."

    def __str__(self):
        return f"Payment - Order ID: {self.order.order_id}, Amount: ${self.amount:.2f}, Method: {self.payment_method}, Status: {self.status}"

In [ ]:
#a function that allows you to query the database for completed orders.
def fetch_completed_orders():
    conn = mysql.connector.connect(
        host='localhost',
        user='your_actual_username',  # Replace with your actual MySQL username
        password='',  # Empty string if no password
        database='restaurant_db'
    )
    cursor = conn.cursor()

    cursor.execute('''SELECT orders.order_id, orders.total, orders.payment_method, orders.status, 
                      GROUP_CONCAT(order_items.item_name, ', ') as items, orders.table_number
                      FROM orders
                      JOIN order_items ON orders.order_id = order_items.order_id
                      WHERE orders.status = 'Paid'
                      GROUP BY orders.order_id''')

    orders = cursor.fetchall()
    conn.close()

    return orders

In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector

class RestaurantApp:
    def __init__(self, root, restaurant):
        self.root = root
        self.restaurant = restaurant
        self.current_order = None
        self.setup_ui()

    def setup_ui(self):
        self.root.title(self.restaurant.name)

        # Set up the Notebook widget for tabs
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill='both')

        # Create the "Restaurant Tab" tab
        self.main_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.main_frame, text="Restaurant Tab")

        # Create the "Completed Orders" tab
        self.orders_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.orders_frame, text="Completed Orders")

        self.setup_main_tab()

        self.setup_orders_tab()

    def setup_main_tab(self):
        # Menu Frame
        self.menu_frame = tk.LabelFrame(self.main_frame, text="Menu", padx=10, pady=10)
        self.menu_frame.grid(row=0, column=0, padx=10, pady=10)
        
        self.menu_listbox = tk.Listbox(self.menu_frame, height=10, width=50)
        self.menu_listbox.pack()

        self.add_menu_items()

        # Order Frame
        self.order_frame = tk.LabelFrame(self.main_frame, text="Order", padx=10, pady=10)
        self.order_frame.grid(row=0, column=1, padx=10, pady=10)

        self.order_listbox = tk.Listbox(self.order_frame, height=10, width=50)
        self.order_listbox.pack()

        # Payment Method Frame
        self.payment_method_frame = tk.LabelFrame(self.main_frame, text="Payment Method", padx=10, pady=10)
        self.payment_method_frame.grid(row=1, column=0, padx=10, pady=10)

        # Payment Method Dropdown
        self.payment_method_var = tk.StringVar(value="Cash")
        self.payment_method_dropdown = ttk.Combobox(
            self.payment_method_frame, 
            textvariable=self.payment_method_var,
            values=["Cash", "Credit Card", "Digital Payment"],
            state="readonly"
        )
        self.payment_method_dropdown.pack()

        # Table Selection Frame
        self.table_selection_frame = tk.LabelFrame(self.main_frame, text="Select Table", padx=10, pady=10)
        self.table_selection_frame.grid(row=1, column=1, padx=10, pady=10)

        # Table Dropdown
        self.table_number_var = tk.StringVar()
        table_numbers = [str(table.table_number) for table in self.restaurant.tables]
        self.table_dropdown = ttk.Combobox(
            self.table_selection_frame, 
            textvariable=self.table_number_var,
            values=table_numbers,
            state="readonly"
        )
        self.table_dropdown.set(table_numbers[0])  # Set default table
        self.table_dropdown.pack()

        # Buttons Frame
        self.buttons_frame = tk.Frame(self.main_frame)
        self.buttons_frame.grid(row=2, column=0, columnspan=2, pady=10)

        self.add_to_order_button = tk.Button(self.buttons_frame, text="Add to Order", command=self.add_to_order)
        self.add_to_order_button.grid(row=0, column=0, padx=10)

        self.remove_from_order_button = tk.Button(self.buttons_frame, text="Remove from Order", command=self.remove_from_order)
        self.remove_from_order_button.grid(row=0, column=1, padx=10)

        self.process_payment_button = tk.Button(self.buttons_frame, text="Process Payment", command=self.process_payment)
        self.process_payment_button.grid(row=0, column=2, padx=10)

        self.clear_order_button = tk.Button(self.buttons_frame, text="Clear Order", command=self.clear_order)
        self.clear_order_button.grid(row=0, column=3, padx=10)

        # Table Frame
        self.table_frame = tk.LabelFrame(self.main_frame, text="Tables", padx=10, pady=10)
        self.table_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

        self.table_status_label = tk.Label(self.table_frame, text="")
        self.table_status_label.pack()

        self.update_table_status()

    def setup_orders_tab(self):
        # Set up a treeview for displaying orders
        self.orders_tree = ttk.Treeview(self.orders_frame, columns=("Order ID", "Table Number", "Total", "Payment Method", "Items"), show="headings")
        self.orders_tree.heading("Order ID", text="Order ID")
        self.orders_tree.heading("Table Number", text="Table Number")
        self.orders_tree.heading("Total", text="Total")
        self.orders_tree.heading("Payment Method", text="Payment Method")
        self.orders_tree.heading("Items", text="Items")
        
        self.orders_tree.column("Order ID", width=80)
        self.orders_tree.column("Table Number", width=100)
        self.orders_tree.column("Total", width=100)
        self.orders_tree.column("Payment Method", width=150)
        self.orders_tree.column("Items", width=300)

        self.orders_tree.pack(expand=True, fill='both', padx=10, pady=10)

        # Load completed orders when the tab is selected
        self.notebook.bind("<<NotebookTabChanged>>", self.on_tab_change)

    def on_tab_change(self, event):
        # Detect when the "Completed Orders" tab is selected
        if self.notebook.tab(self.notebook.select(), "text") == "Completed Orders":
            self.show_completed_orders()

    def show_completed_orders(self):
        orders = fetch_completed_orders()
        # Clear previous content in the treeview
        for i in self.orders_tree.get_children():
            self.orders_tree.delete(i)
        # Insert new data
        for order in orders:
            self.orders_tree.insert("", "end", values=(order[0], order[5], f"${order[1]:.2f}", order[2], order[4]))

    def add_menu_items(self):
        self.menu_listbox.delete(0, tk.END)
        for item in self.restaurant.menu:
            self.menu_listbox.insert(tk.END, f"{item.name} - {item.category}: ${item.price:.2f}")

    def add_to_order(self):
        selected_item_index = self.menu_listbox.curselection()
        if selected_item_index:
            item_name = self.menu_listbox.get(selected_item_index).split(" - ")[0]
            if self.current_order is None:
                self.current_order = Order()
            self.restaurant.find_menu_item_by_name(item_name, self.current_order)
            self.update_order_listbox()
        else:
            messagebox.showwarning("Selection Error", "Please select a menu item to add.")

    def remove_from_order(self):
        selected_item_index = self.order_listbox.curselection()
        if selected_item_index and self.current_order:
            item_name = self.order_listbox.get(selected_item_index).split(" - ")[0]
            item = next((item for item in self.current_order.items if item.name == item_name), None)
            if item:
                self.current_order.remove_item(item)
                self.update_order_listbox()
        else:
            messagebox.showwarning("Selection Error", "Please select an item to remove from the order.")

    def process_payment(self):
        if self.current_order and self.current_order.items:
            payment_method = self.payment_method_var.get()
            table_number = int(self.table_number_var.get())  # Get selected table number
            self.current_order.table_number = table_number  # Assign the table number to the order
            self.restaurant.assign_table(table_number, self.current_order)
            payment_result = self.restaurant.process_payment(table_number, payment_method)
            messagebox.showinfo("Payment", payment_result)
            receipt = self.restaurant.print_receipt(table_number)
            messagebox.showinfo("Receipt", receipt)
            self.current_order = None
            self.update_order_listbox()
            self.update_table_status()
        else:
            messagebox.showwarning("Order Error", "No items in the order to process payment.")

    def clear_order(self):
        self.current_order = None
        self.update_order_listbox()

    def update_order_listbox(self):
        self.order_listbox.delete(0, tk.END)
        if self.current_order:
            for item in self.current_order.items:
                self.order_listbox.insert(tk.END, f"{item.name} - {item.category}: ${item.price:.2f}")

    def update_table_status(self):
        status = "\n".join([str(table) for table in self.restaurant.tables])
        self.table_status_label.config(text=status)

In [ ]:
if __name__ == "__main__":
    restaurant = Restaurant("Gourmet Dining")
    restaurant.add_menu_item(MenuItem("Pasta", 10.99, "Food"))
    restaurant.add_menu_item(MenuItem("Steak", 24.99, "Food"))
    restaurant.add_menu_item(MenuItem("Coffee", 8.99, "Drink"))
    restaurant.add_menu_item(MenuItem("Pizza", 6.99, "Food"))
    restaurant.add_menu_item(MenuItem("Orange Juice", 2.99, "Drink"))
    restaurant.add_menu_item(MenuItem("Cheesecake", 7.99, "Dessert"))
    restaurant.add_menu_item(MenuItem("Pudding", 4.49, "Dessert"))
    restaurant.add_menu_item(MenuItem("Ice Cream", 4.99, "Dessert"))
    restaurant.add_menu_item(MenuItem("Hot Chocolate", 4.49, "Drink"))
    restaurant.add_menu_item(MenuItem("Milkshake", 6.99, "Drink"))
    restaurant.add_menu_item(MenuItem("Espresso", 3.99, "Drink"))
    restaurant.add_menu_item(MenuItem("Mojito", 7.99, "Drink"))
    restaurant.add_menu_item(MenuItem("Burger", 12.99, "Food"))
    restaurant.add_menu_item(MenuItem("Salad", 7.99, "Food"))
    restaurant.add_menu_item(MenuItem("Fries", 4.99, "Food"))

    restaurant.add_table(Table(1))
    restaurant.add_table(Table(2))
    restaurant.add_table(Table(3))
    restaurant.add_table(Table(4))

    restaurant.add_staff(Staff("John", "Waiter"))
    restaurant.add_staff(Staff("Emily", "Chef"))

    # Start the GUI
    root = tk.Tk()
    app = RestaurantApp(root, restaurant)
    root.mainloop()

In [ ]:
# Example usage to print all completed orders
completed_orders = fetch_completed_orders()
for order in completed_orders:
    print(f"Order ID: {order[0]}, Total: {order[1]:.2f}, Payment Method: {order[2]}, Items: {order[4]}")